In [83]:
# Imports
import os
import time
import glob
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from imutils.video import VideoStream
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import Bunch
import pandas as pd
from functions import *


In [84]:
# Constants
data_path = 'images'
mappings = {"r": "r", "p": "p", "s": "s"}
frame_size = (320, 240)
feature_names = [
    'area',
    'perimeter',
    'aspect_ratio',
    'extent',
    'convex_hull_area',           # Area of the convex hull
    'hull_perimeter',              # Perimeter of the convex hull
    'solidity',                    # Ratio of area to convex hull area
    'circularity',                 # Measure of how circular the shape is
    'eccentricity',                # Measure of the elongation of the shape
    'num_convexity_defects',       # Number of convexity defects
    'max_defect_depth',            # Maximum depth of convexity defects
    'major_axis_length',           # Length of the major axis of the fitted ellipse
    'minor_axis_length',           # Length of the minor axis of the fitted ellipse
    'orientation',                  # Orientation angle of the fitted ellipse
    'hu_moment',
    'centroid_x',
    'centroid_y'
]


categories = ['Rock', 'Paper', 'Scissor']
size = (320, 240)

lower_skin = np.array([0, 70, 40], dtype=np.uint8)  # Example lower boundary of skin tone in HLS
upper_skin = np.array([30, 200, 255], dtype=np.uint8)  # Example upper boundary of skin tone in HLS




In [85]:
# Initialize lists
data = np.empty((0, len(feature_names)), float)  # To store feature vectors
target = []  # To store labels
images = []  # List to store images
labels = []  # List to store corresponding labels

In [86]:
# Key mappings
kMappings = {ord(key): value for key, value in mappings.items()}

In [88]:
for category in categories:
    # Get the corresponding label from the mappings dictionary
    label = mappings[category[0].lower()]  # 'r' -> 0, 'p' -> 1, 's' -> 2
    
    # Form the path to the current category's images
    folder_path = os.path.join(data_path, category)
    
    # Get all image file paths in the current folder
    image_paths = glob.glob(os.path.join(folder_path, '*.jpg'))
    
    print(f"[INFO] Processing {category} images, found: {len(image_paths)}")
    
    # Loop over each image path
    for image_path in image_paths:
        # Load the image using OpenCV
        image = cv.imread(image_path)
        
        if image is None:
            print(f"[WARNING] Unable to load image {image_path}. Skipping.")
            continue
        
        # Resize the image to a standard frame size
        image_resized = cv.resize(image, frame_size)
        
        # Append the image and its label
        images.append(image_resized)  # Store the image
        labels.append(label)  # Store the corresponding label

# Convert images and labels to NumPy arrays for consistency
images = np.array(images)
labels = np.array(labels)

# Display some information about the dataset
print(f"[INFO] Number of images loaded: {len(images)}")
print(f"[INFO] Shape of images array: {images.shape}")
print(f"[INFO] Number of labels: {len(labels)}")

[INFO] Processing Rock images, found: 54
[INFO] Processing Paper images, found: 40
[INFO] Processing Scissor images, found: 39
[INFO] Number of images loaded: 133
[INFO] Shape of images array: (133, 240, 320, 3)
[INFO] Number of labels: 133


In [89]:
# Loop over each image and process for background removal
for i, image in enumerate(images):
    # Convert the image to the HSV color space
    img_hls = cv.cvtColor(image, cv.COLOR_BGR2HLS)

    # Create a binary mask where skin color is within the specified range
    mask = cv.inRange(img_hls, lower_skin, upper_skin)

    # Apply morphological operations to clean up the mask (remove small noises)
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    mask = cv.erode(mask, None, iterations=2)
    mask = cv.dilate(mask, None, iterations=2)

    # Create the result image by applying the mask to the original image
    result = cv.bitwise_and(image, image, mask=mask)

    # Convert the result to grayscale to proceed with feature extraction
    img_gray = cv.cvtColor(result, cv.COLOR_BGR2GRAY)

    # Apply Gaussian blur to the grayscale image to reduce noise
    img_blurred = cv.GaussianBlur(img_gray, (5, 5), 0)

    # Apply binary thresholding to convert the image to binary (black & white)
    img_BW = cv.adaptiveThreshold(img_blurred, 255, cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY_INV, 11, 2)
    # _, img_BW = cv.threshold(img_blurred, 110, 255, cv.THRESH_BINARY_INV)

    # Apply morphological operations to remove small noise and close gaps
    img_BW = cv.morphologyEx(img_BW, cv.MORPH_CLOSE, kernel)

    # Check if any foreground remains after thresholding
    if cv.countNonZero(img_BW) == 0:
        print(f"[WARNING] No foreground detected in image {i}. Skipping.")
        continue

    # Find the largest contour from the binary image
    try:
        contour = getLargestContour(img_BW)
    except ValueError:
        print(f"[WARNING] Could not find contours in image {i}. Skipping.")
        continue

    # Extract features from the largest contour
    features = getAdvancedContourFeatures(contour)
    
    # Append the features to the data matrix
    data = np.append(data, np.array([features]), axis=0)
    
    # Append the corresponding label from the 'labels' array
    target.append(labels[i])

        # draw the outline of the object
    cv.drawContours(image, [contour], -1, (0, 255, 0), 2)
    cv.drawContours(mask, [contour], -1, (0, 255, 0), 2)
    cv.drawContours(result, [contour], -1, (0, 255, 0), 2)
    cv.drawContours(img_BW, [contour], -1, (0, 255, 0), 2)

    # point out hull defects
    # if defects is not None:
    #     for s,e,f,d in defects:
    #         start = tuple(contour[s])
    #         end = tuple(contour[e])
    #         far = tuple(contour[f])
    #         cv.line(image,start,end,[0,255,255],2)
    #         cv.circle(image,far,5,[0,0,255],-1)

    # Optionally, display the original image, mask, and result for visualization
    cv.imshow("HSV Image", img_hls)
    cv.imshow("Original Image", image)
    cv.imshow("Skin Mask", mask)
    cv.imshow("Result after Background Removal", result)
    cv.imshow("Binary Image for Contour", img_BW)

    # Wait for a key press and quit if 'q' is pressed
    k = cv.waitKey(1000) & 0xFF
    if k == ord("q"):
        break

# Convert the target list to a NumPy array
target = np.array(target)

# Display the shape of the resulting feature dataset
print(f"[INFO] Feature extraction complete.")
print(f"[INFO] Shape of data matrix: {data.shape}")
print(f"[INFO] Number of target labels: {target.shape[0]}")

# Create a Bunch object containing the dataset
dataset = Bunch(data=data,
                target=target,
                feature_names=feature_names,
                unique_targets=np.unique(target))

print(f"[INFO] Dataset created with shape: {dataset.data.shape}")

In [81]:
unpackAdvancedFeatures(dataset.data, dataset.target)

[INFO] Number of feature vectors: 133
[INFO] Number of labels: 133
Image #   Area           Perimeter      Aspect Ratio   Extent         Convex Hull Area    Hull Perimeter      Solidity       Circularity    Eccentricity   Convexity Defects   Max Defect Depth    Major Axis Length   Minor Axis Length   Orientation    Hu moments     Centroid_x     Centroid_y     Label
0         5640.00        2093.01        2.02           0.20           19362.50            591.50              0.29           0.02           0.91           16.0                231.00              220.34              92.99               100.33         0.01           220.00         157.00         r
1         16918.00       836.98         2.29           0.74           19092.50            582.47              0.89           0.30           0.90           17.0                35.00               235.45              100.86              92.79          0.62           198.00         137.00         r
2         17844.50       1080.42      

In [82]:
# Export to csv
"""Export the feature data and target labels to a CSV file."""
# Create a DataFrame from the feature data
f = pd.DataFrame(data, columns=feature_names)  # Use the feature_names list
# Ad the target labels as a new column
f['Label'] = target
# Expot to CSV
filename = 'data/hand_gesture_features.csv'
f.to_csv(filename, index=False)
print(f"[INFO] Data exported to {filename}")

[INFO] Data exported to data/hand_gesture_features.csv
